In [1]:
def sph2cart(r,theta,phi):
    rcos_theta = r * np.cos(theta)
    x = rcos_theta * np.cos(phi)
    y = rcos_theta * np.sin(phi)
    z = r * np.sin(theta)
    return x, y, z

def cart2sph(x,y,z):
    XsqPlusYsq = x**2 + y**2
    r = math.sqrt(XsqPlusYsq + z**2)               # r
    elev = math.atan2(z,math.sqrt(XsqPlusYsq))     # theta
    az = math.atan2(y,x)                           # phi
    return r, elev, az

In [2]:
from PIL import Image
import numpy as np
import math
import open3d as o3d
from open3d import JVisualizer
from multiprocessing.pool import Pool
import pymesh
import os

In [3]:
img = Image.open("/Users/rjpj/downloads/DM/Paper lantern_Depth.png")
img_bak = img

In [4]:
# Lower depth map resolution
nH = img.height//10
nW = img.width//10
img = img.resize((nW,nH),Image.ANTIALIAS)

In [5]:
img.show()

In [6]:
I = np.asarray(img)

In [7]:
I.shape
height,width,channels = I.shape
center_x = width//2
center_y = height//2

In [8]:
I[0].shape

(108, 4)

In [10]:
pool = Pool(3)
def ptGen(tup):
    x,y,z,r = tup
    max_r , theta , phi = cart2sph(x,y,z)
    r_normalized = (r/256) * max_r
    x,y,z = sph2cart(r_normalized,theta,phi)
    return [x,y,z]
    
queries = []
for i in range(height):
    for j in range(width):
        x = j - center_x
        y = center_y - i
        z = 200
        r = I[i][j][0]
        
        queries.append((x,y,z,r))  
        
pt_cloud = pool.map( ptGen , queries)
pool.close()

In [11]:
pt_cloud[:10]

[[-49.35937499999999, 87.75, 182.8125],
 [-48.445312499999986, 87.75000000000003, 182.8125],
 [-47.531250000000014, 87.75000000000003, 182.8125],
 [-46.61718750000001, 87.75000000000003, 182.8125],
 [-45.703124999999986, 87.75000000000001, 182.8125],
 [-44.789062500000014, 87.75, 182.81249999999997],
 [-43.875000000000014, 87.75000000000003, 182.8125],
 [-42.777343750000014, 87.375, 182.03125],
 [-41.86718750000001, 87.37500000000003, 182.03125],
 [-40.95703125, 87.37500000000001, 182.03125]]

In [12]:
pt_cloud[:10]

[[-49.35937499999999, 87.75, 182.8125],
 [-48.445312499999986, 87.75000000000003, 182.8125],
 [-47.531250000000014, 87.75000000000003, 182.8125],
 [-46.61718750000001, 87.75000000000003, 182.8125],
 [-45.703124999999986, 87.75000000000001, 182.8125],
 [-44.789062500000014, 87.75, 182.81249999999997],
 [-43.875000000000014, 87.75000000000003, 182.8125],
 [-42.777343750000014, 87.375, 182.03125],
 [-41.86718750000001, 87.37500000000003, 182.03125],
 [-40.95703125, 87.37500000000001, 182.03125]]

In [13]:
pt_arr= np.array(pt_cloud)
pt_arr.shape

(20736, 3)

In [14]:
o3d_cloud = o3d.geometry.PointCloud()

In [15]:
o3d_cloud.points = o3d.utility.Vector3dVector(pt_arr)

In [16]:
o3d.visualization.draw_geometries_with_editing([o3d_cloud])

In [17]:
len(pt_cloud)

20736

In [18]:
texture_verts = []
mesh = []

for i in range(height):
    for j in range(width):
        tex = [
            j / width,
            1 - i / height
        ]
        texture_verts.append(tex)

for i in range(height - 1 ):
    for j in range(width - 1):
        triangle1 = [
            i*width + j  ,
            i*width + (j + 1) ,
            (i+1)* width + j
        ]
        triangle1.reverse()
        
        triangle2 = [
            i*width + (j + 1) ,
            (i+1) * width + (j+1),
            (i+1)* width + j
        ]
        triangle2.reverse()
        
        mesh.append(triangle1)
        mesh.append(triangle2)

        

In [19]:
mesh_arr = np.array(mesh)

In [20]:
mesh_arr.shape

(40874, 3)

In [21]:
o3d_mesh = o3d.geometry.TriangleMesh()

In [22]:
o3d_mesh.vertices= o3d.utility.Vector3dVector(pt_arr)

In [23]:
o3d_mesh.triangles = o3d.utility.Vector3iVector(mesh_arr)

In [24]:
o3d.visualization.draw_geometries_with_editing([o3d_mesh])

In [25]:
o3d_lines = o3d.geometry.LineSet.create_from_triangle_mesh(o3d_mesh)

In [26]:
o3d.visualization.draw_geometries_with_editing([o3d_lines])

In [115]:
temp = o3d_mesh.filter_smooth_taubin()

In [116]:
o3d.visualization.draw_geometries_with_editing([temp])

In [358]:
def writeObj(verts,faces,fName="test.obj"):
    f = open(fName,"w+")
    for vert in verts:
        x,y,z = vert
        f.write("v {} {} {}\n".format(x/1000,y/1000,z/1000))
        
    for t_vert in texture_verts:
        u,v = t_vert
        f.write("vt {} {}\n".format(u,v))
    
    for face in faces:
        a,b,c = face
        f.write("f {}/{} {}/{} {}/{}\n".format(a+1,a+1,b+1,b+1,c+1,c+1))
    f.close()
        

In [359]:
writeObj(pt_arr,mesh_arr)

In [128]:
os.getcwd()

'/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/Resources/Python.app/Contents/Resources'

In [129]:
mesh_arr.shape

(4141202, 3)

In [130]:
pt_arr.shape

(2073600, 3)

In [364]:
t = Image.open("/Users/rjpj/downloads/DM/Paper lantern.png")


In [365]:
t.show()

In [367]:
t_r = t.resize((1920,1920),Image.ANTIALIAS)

In [368]:
t_r.save("/Users/rjpj/downloads/DM/Paper lantern square.png")

In [369]:
t_r.show()